In [10]:
import requests
import gradio as gr

# Ollama API setup
OLLAMA_API = "http://localhost:11434/api/chat"
OLLAMA_MODEL = "llama3.2"

# Check if Ollama is running
try:
    response = requests.get("http://localhost:11434/api/version")
    if response.status_code == 200:
        print(f"Ollama is running. Version: {response.json().get('version', 'unknown')}")
    else:
        print("Ollama is available but returning unexpected response")
except requests.exceptions.ConnectionError:
    print("Warning: Ollama server not detected at http://localhost:11434")
    print("Make sure Ollama is running before using this application")

# System prompt
system_message = """You are a helpful assistant in a clothes store. You should try to gently encourage 
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. 
For example, if the customer says 'I'm looking to buy a hat', 
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'
Encourage the customer to buy hats if they are unsure what to get.
If the customer asks for shoes, you should respond that shoes are not on sale today, 
but remind the customer to look at hats!
"""

def chat(message, history):
    # Add context if belts are mentioned
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."

    # Prepare messages for Ollama API
    messages = [{"role": "system", "content": relevant_system_message}]
    messages.extend(history) #all previous messages (history) in OpenAI-style roles: "user" / "assistant"
    messages.append({"role": "user", "content": message})

    # Send POST request to Ollama's chat endpoint
    try:
        response = requests.post(OLLAMA_API, json={
            "model": OLLAMA_MODEL,
            "messages": messages,
            "stream": False
        })

        if response.status_code == 200:
            reply = response.json()["message"]["content"]
            yield reply
        else:
            yield f"Error: Received status code {response.status_code} from Ollama."

    except requests.exceptions.RequestException as e:
        yield f"Request failed: {e}"

# Launch Gradio UI
gr.ChatInterface(fn=chat, type="messages").launch()


Ollama is running. Version: 0.6.5
* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
